# Notice 

본 jupyter notebook 파일은 "" 최종 프로젝트 과제 코드입니다.  
셀 순서대로 실행하시면 됩니다. 
 
- 프로젝트 제목: 해외 뉴스 매체 기반 영어 단어장 만들기  
- 작성자: "dolphin"
- 참고자료
    - [출처] 파이썬#70 - 파이썬 크롤링과 nltk 를 활용한 영어단어 퀴즈 게임|작성자 남박사(https://blog.naver.com/nkj2001/222713831884)
    - [출처] Python을 통한 BBC 뉴스 크롤링하기(https://1eed00.tistory.com/175)
    - [출처] 딥 러닝을 이용한 자연어 처리 입문(https://wikidocs.net/22530)

# 0. Libraries

본 프로젝트에서는 아래의 Python library를 사용하였습니다.  
필요에 의해 !pip install 구문으로 설치할 수 있습니다.  
nltk 라이브러리의 경우 원활하게 사용하기 위해 자연어 데이터를 다운로드해야 합니다. 다운로드 코드는 주석 처리를 해두었습니다. 

- Requests, BeautifulSoup (크롤링을 위해)
- nltk (자연어 데이터 처리를 위해)
- Pandas (데이터 처리를 위해)
- re, random, string

In [42]:
import requests
from bs4 import BeautifulSoup
import nltk 
# nltk.download() # nltk data를 다운받기 위한 코드
import pandas as pd
import re, random, string

# 1. International News Crawling 
해외 뉴스 매체 3곳(ABC, BBC, CNN)으로부터 영단어를 크롤링합니다. 
- abc_df: ABC News 데이터
- bbc_df: BBC News 데이터
- cnn_df: CNN News 데이터
- all_df: abc_df + bbc_df + cnn_df (크롤링된 News 데이터를 모두 모은 데이터프레임)

In [39]:
# 뉴스 매체 URL 설정
abc_url = 'https://abcnews.go.com/International'
bbc_url = 'http://feeds.bbci.co.uk/news/rss.xml'
cnn_url = 'https://www.cnn.com'

In [40]:
# 헤더 설정
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'}

In [56]:
# CNN news 메인 페이지에서 기사 링크에 접근하는 과정에서 쓰이는 함수
def url_is_article(url, current_year='2023'):
    if url:
        if 'cnn.com/{}/'.format(current_year) in url and '/gallery/' not in url:
            return True
    return False

# 크롤링 과정 중 접근한 요소에 text 유무를 탐지하는 함수 (error handling)
def return_text_if_not_none(element):
    if element:
        return element.text.strip()
    else:
        return ''

### ABC News 

In [53]:
# 뉴스 메인 페이지 접근
req = requests.get(abc_url, headers=header)
bs = BeautifulSoup(req.text, "html.parser")

# 뉴스 메인에 있는 기사 링크에 하나 씩 접근
all_data = []
for link in bs.select("div.ContentList__Item > a"): 
    req = requests.get(link.get("href"), headers=header)
    bs = BeautifulSoup(req.text, "html.parser")
    
    # 기사 제목, 기사 내용 크롤링
    title = return_text_if_not_none(bs.find('h1'))
    contents = return_text_if_not_none(bs.find('div', attrs={'data-testid': 'prism-article-body'}))
    content = [i.text.strip() for i in bs.find_all('p')]
    content = ' '.join(content)

    data = [title, content, "ABC"]
    all_data.append(data)

# 기사 제목, 기사 내용, 뉴스 매체 종류 -> 총 3개의 컬럼을 가진 dataframe형태로 크롤링 데이터 저장
abc_df = pd.DataFrame(all_data, columns=['Title', 'Article Text', 'News'])
abc_df.head()

,Title,Article Text,News
0,"Missing at age 11, found six years later in Fr...","There's still a heap of unanswered questions, ...",ABC
1,Cardinal sentenced to 5 1/2 years in Vatican's...,Cardinal Becciu and 8 others were convicted of...,ABC
2,"Vatican 'trial of the century,' a Pandora's bo...",It is the most complicated financial trial in ...,ABC
3,61 migrants drown in 'shipwreck' off Libyan co...,The vessel left Libya with about 86 people onb...,ABC
4,The Indian Navy is shadowing a bulk carrier li...,The Indian Navy says it is shadowing a bulk ca...,ABC


### BBC News

In [49]:
# 뉴스 메인 페이지 접근
req = requests.get(bbc_url, headers=header)
bs = BeautifulSoup(req.content.decode('utf-8'), "xml") # BBC news의 경우 xml 페이지에서 parsing하기 때문에 encoding-decoding에 유의해야 함

# 뉴스 메인에 있는 기사 링크에 하나 씩 접근
all_data = []
for item in bs.find_all('item'): # item과 그 내의 자식요소만 필터링
    link = item.find('guid').get_text()
    req = requests.get(link, headers=header)
    bs = BeautifulSoup(req.content.decode('utf-8'), "xml")
    
    # 기사 제목, 기사 내용 크롤링
    title = item.find_all(["title","description"])[0].get_text()
    content = [i.text.strip() for i in bs.find_all('p')]
    content = ' '.join(content)
    
    data = [title, content, "BBC"] 
    all_data.append(data)

# 기사 제목, 기사 내용, 뉴스 매체 종류 -> 총 3개의 컬럼을 가진 dataframe형태로 크롤링 데이터 저장
bbc_df = pd.DataFrame(all_data, columns=['Title', 'Article Text', 'News'])
bbc_df.head()

,Title,Article Text,News
0,Michelle Mone admits she stands to benefit fro...,This video can not be played Michelle Mone: I'...,BBC
1,British teen Alex Batty returns to UK after si...,"British teenager Alex Batty, who was found in ...",BBC
2,Ian Wright: Match of the Day pundit to step do...,,BBC
3,Why Covid is still flooring some people,What is it like to catch Covid now? It is a qu...,BBC
4,Strictly Come Dancing 2023: Winner of glitterb...,This video can not be played Watch: Moment win...,BBC


### CNN News

In [62]:
# 뉴스 메인 페이지 접근
req = requests.get(cnn_url).text
bs = BeautifulSoup(req, features="html.parser")

# 뉴스 메인에 있는 기사 링크 수집
all_urls = []
for a in bs.find_all('a', href=True):
    if a['href'] and a['href'][0] == '/' and a['href'] != '#':
        a['href'] = cnn_url + a['href']
    all_urls.append(a['href'])
article_urls = [url for url in all_urls if url_is_article(url)]

# 뉴스 메인에 있는 기사 링크에 하나 씩 접근
all_data = []
article_urls_duplicates_removed = list(set(article_urls)) # 중복 URL 제거 
for link in article_urls_duplicates_removed:
    req = requests.get(link).text
    bs = BeautifulSoup(req, features="html.parser")
    
    # 기사 제목, 기사 내용 크롤링
    title = return_text_if_not_none(bs.find('h1', {'class': 'headline__text'}))
    content = return_text_if_not_none(bs.find('div', {'class': 'article__content'}))
    
    data = [title, content, "CNN"] 
    all_data.append(data)
    
# 기사 제목, 기사 내용, 뉴스 매체 종류 -> 총 3개의 컬럼을 가진 dataframe형태로 크롤링 데이터 저장
cnn_df = pd.DataFrame(all_data, columns=['Title', 'Article Text', 'News'])
cnn_df.head()

,Title,Article Text,News
0,Sharon Osbourne says getting surgery on her fa...,London\nCNN\n — \n \n\n\nSharon Osb...,CNN
1,Weeks-old government dubbed ‘anti-Māori’ as cu...,CNN\n — \n \n\n\n New Zealand’...,CNN
2,"Hundreds of objects in British Museum defaced,...",London\nCNN\n — \n \n\n\n Hund...,CNN
3,Here’s where the minimum wage will increase ne...,New York\nCNN\n — \n \n\n\n At...,CNN
4,Peeling back the layers of the extraordinary v...,CNN\n — \n \n\n\n At this very...,CNN


### Concate [ABC, BBC, CNN]

In [65]:
all_df = pd.concat([abc_df, bbc_df, cnn_df]).reset_index(drop=True) 
all_df = all_df.astype('string') # text형태의 데이터이기 때문에 형식 변환이 필요함

print("수집된 ABC News 기사 수: ", len(abc_df))
print("수집된 BBC News 기사 수: ", len(bbc_df))
print("수집된 CNN News 기사 수: ", len(cnn_df))
print("수집된 전체 News 기사 수: ", len(all_df))

수집된 ABC News 기사 수:  9
수집된 BBC News 기사 수:  39
수집된 CNN News 기사 수:  73
수집된 전체 News 기사 수:  121


# 2. Preprocessing Natural Language Data
크롤링한 기사 데이터는 text 형태의 자연어 데이터입니다.  
띄어쓰기, 개행 처리 등 필요하지 않은 부분들을 처리하고 nltk 라이브러리를 이용하여 단어 별로 분리하는 작업을 진행합니다. 

### Basic Preprocssing 

In [80]:
# 개행 문자(엔터), 공란(스페이스바), 크롤링 과정 중 잘못 가져온 값들을 직접 대체(replace)하여 처리
all_df['Article Text'] = all_df['Article Text'].str.split().str.join(' ')
all_df['Article Text'] = all_df['Article Text'].apply(lambda x: str(x).replace(u'\xa0', u''))
all_df['Article Text'] = all_df['Article Text'].apply(lambda x: str(x).replace('\n', ''))
all_df['Article Text'] = all_df['Article Text'].apply(lambda x: str(x).replace('\'', ''))
all_df['Article Text'] = all_df['Article Text'].apply(lambda x: str(x).replace('ABC', ''))
all_df['Article Text'] = all_df['Article Text'].apply(lambda x: str(x).replace('BBC', ''))
all_df['Article Text'] = all_df['Article Text'].apply(lambda x: str(x).replace('CNN', ''))
all_df['Article Text'] = all_df['Article Text'].astype('string')

In [81]:
# 기본 전처리를 끝낸 데이터프레임에서 기사 글을 하나의 text 형태로 추출
text = all_df['Article Text'].values
text = ' '.join(text)
print(text[:500])

Theres still a heap of unanswered questions, notably: Where has he been? LE PECQ, France -- The vehicles headlights silhouetted the exhausted teenager walking alone in the rain in deepest rural France, with a skateboard tucked under his arm. “I said to myself, ‘That’s strange. It’s 3 am in the morning, it’s raining, he’s all by himself on the road between two villages," delivery driver Fabien Accidini recounted. From there, the story gets stranger still. The youngster, it turned out, was Alex Ba


### Preprocessing with NLTK
1. 토크나이징(토큰화)
2. 불용어 제거 
3. 표제어 추출 
4. 품사 태깅

In [82]:
# 토크나이징: 텍스트에 대해 특정 기준(단어 기준)으로 문장을 나누는 것
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(text) # 토큰화된 단어들은 tokens 변수에 저장
print(tokens[:100])

['Theres', 'still', 'a', 'heap', 'of', 'unanswered', 'questions', ',', 'notably', ':', 'Where', 'has', 'he', 'been', '?', 'LE', 'PECQ', ',', 'France', '--', 'The', 'vehicles', 'headlights', 'silhouetted', 'the', 'exhausted', 'teenager', 'walking', 'alone', 'in', 'the', 'rain', 'in', 'deepest', 'rural', 'France', ',', 'with', 'a', 'skateboard', 'tucked', 'under', 'his', 'arm.', '“I', 'said', 'to', 'myself', ',', '‘That’s', 'strange.', 'It’s', '3', 'am', 'in', 'the', 'morning', ',', 'it’s', 'raining', ',', 'he’s', 'all', 'by', 'himself', 'on', 'the', 'road', 'between', 'two', 'villages', ',', "''", 'delivery', 'driver', 'Fabien', 'Accidini', 'recounted.', 'From', 'there', ',', 'the', 'story', 'gets', 'stranger', 'still.', 'The', 'youngster', ',', 'it', 'turned', 'out', ',', 'was', 'Alex', 'Batty', ',', 'a', '17-year-old', 'from']


In [83]:
# 불용어(stop words) 제거: 유의미하지 않은 단어(token)를 제거하는 것 (ex, 조사, 접미사, I, my, me, mine...) 
stop_words = nltk.corpus.stopwords.words('english') + list(string.punctuation)
clean_tokens = [token for token in tokens if token.lower() not in stop_words]
print(clean_tokens[:100])

['Theres', 'still', 'heap', 'unanswered', 'questions', 'notably', 'LE', 'PECQ', 'France', '--', 'vehicles', 'headlights', 'silhouetted', 'exhausted', 'teenager', 'walking', 'alone', 'rain', 'deepest', 'rural', 'France', 'skateboard', 'tucked', 'arm.', '“I', 'said', '‘That’s', 'strange.', 'It’s', '3', 'morning', 'it’s', 'raining', 'he’s', 'road', 'two', 'villages', "''", 'delivery', 'driver', 'Fabien', 'Accidini', 'recounted.', 'story', 'gets', 'stranger', 'still.', 'youngster', 'turned', 'Alex', 'Batty', '17-year-old', 'Britain', 'missing', 'since', '2017.', 'British', 'French', 'authorities', 'confirmed', 'Friday', 'teenager', 'found', 'Accidini', 'week', 'boy', 'vanished', 'age', '11', 'mother', 'grandfather', 'took', 'meant', 'two-week', 'family', 'holiday', 'Spain.', 'Instead', 'turned', 'six-year', 'odyssey', 'Morocco', 'Spain', 'southwest', 'France', 'living', 'off-the-grid', 'life.', 'week.', 'Batty', 'suddenly', 'popped', 'back', 'radar', 'Wednesday.', 'Thats', 'Accidini', 'fou

In [84]:
# 표제어 추출: 단어들을(tokens) 기본 사전형 단어(표제어)로 변환하는 것 (am, are, is -> be라는 표제어로 추출 가능)
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
lemm_tokens = [wnl.lemmatize(token) for token in clean_tokens]
print(lemm_tokens[:100])

['Theres', 'still', 'heap', 'unanswered', 'question', 'notably', 'LE', 'PECQ', 'France', '--', 'vehicle', 'headlight', 'silhouetted', 'exhausted', 'teenager', 'walking', 'alone', 'rain', 'deepest', 'rural', 'France', 'skateboard', 'tucked', 'arm.', '“I', 'said', '‘That’s', 'strange.', 'It’s', '3', 'morning', 'it’s', 'raining', 'he’s', 'road', 'two', 'village', "''", 'delivery', 'driver', 'Fabien', 'Accidini', 'recounted.', 'story', 'get', 'stranger', 'still.', 'youngster', 'turned', 'Alex', 'Batty', '17-year-old', 'Britain', 'missing', 'since', '2017.', 'British', 'French', 'authority', 'confirmed', 'Friday', 'teenager', 'found', 'Accidini', 'week', 'boy', 'vanished', 'age', '11', 'mother', 'grandfather', 'took', 'meant', 'two-week', 'family', 'holiday', 'Spain.', 'Instead', 'turned', 'six-year', 'odyssey', 'Morocco', 'Spain', 'southwest', 'France', 'living', 'off-the-grid', 'life.', 'week.', 'Batty', 'suddenly', 'popped', 'back', 'radar', 'Wednesday.', 'Thats', 'Accidini', 'found', 'a

In [103]:
# 품사 태깅: 단어(token)에 품사를 붙여주는 작업 
# 품사를 보고 단어 학습에서 제외할 품사(ex, 조사, 접속사..)의 단어들은 제거하기 위해
EXCLUDE = ["CC", "CD", "DT", "EX", "IN", "MD", "POS", "PRP", "PRP$", "UH", "VBD", "VBG", "VBN", "VBZ", "WDT", "WP", "WP$", "WRB"]

tags = nltk.pos_tag(lemm_tokens)
word_dict = {}
for word, tag in tags:
    if word.isalpha() and len(word) > 2 and tag not in EXCLUDE: # 영어이고, 1글자이상이며, 제외 품사에 해당되지 않을 때 
        if word_dict.get(word) is None:
            word_dict[word] = 1
        else:
            word_dict[word] += 1  # 같은 단어를 results dictionary에 넣을 경우 빈도 수 +1 

word_list = sorted(word_dict.items(), key=lambda item: item[1], reverse=True) # 빈도수 순서대로 단어 dictionary 정렬
words = [x[0] for x in word_list] # 오직 단어만 담은 list 
print("크롤링된 단어 사전 길이: ", len(word_list))
print(word_list[:100])

크롤링된 단어 사전 길이:  8989
[('year', 220), ('also', 191), ('people', 180), ('say', 176), ('first', 152), ('Images', 146), ('time', 140), ('back', 128), ('family', 121), ('new', 114), ('get', 109), ('New', 102), ('right', 91), ('still', 87), ('make', 86), ('day', 84), ('world', 82), ('last', 81), ('work', 78), ('call', 77), ('company', 77), ('want', 76), ('chip', 76), ('life', 75), ('even', 75), ('death', 75), ('way', 73), ('support', 72), ('team', 71), ('think', 71), ('see', 70), ('thing', 69), ('war', 69), ('many', 69), ('Vatican', 68), ('Gaza', 68), ('win', 68), ('week', 66), ('woman', 66), ('number', 66), ('Taiwans', 66), ('state', 64), ('take', 64), ('part', 63), ('month', 62), ('show', 62), ('external', 62), ('come', 61), ('home', 60), ('secret', 60), ('Israel', 59), ('game', 59), ('Hamas', 58), ('much', 57), ('talk', 56), ('France', 55), ('really', 55), ('sauce', 55), ('superstardom', 55), ('case', 54), ('plan', 54), ('claim', 54), ('tell', 54), ('help', 53), ('news', 52), ('well', 52)

# 3. Translation with Daum Dictionary 
포털사이트 다음(Daum)에서 찾고자 하는 키워드를 전달하면 한글 뜻과 영어 뜻을 반환하는 함수(get_mean)를 작성하였다. 

In [87]:
from urllib.request import urlopen
def get_mean(search_keyword):
    # daum 사전 사이트에 접근
    url = 'https://alldic.daum.net/search.do?q=' + search_keyword
    web = urlopen(url) 
    web_page = BeautifulSoup(web, 'html.parser')
    
    # daum 사전 사이트에 찾고자 하는 키워드(search_keyword)를 입력하고 나온 결과창의 요소들에 접근
    mean_data = []
    cnt = 0 
    for block in web_page.find_all('div', {'class': 'card_word'}): # 결과창 요소 = block 
        mean = block.find('ul', {'class': 'list_search'}).get_text().replace("\n", " ")
        mean_data.append(mean)
        cnt += 1 
        if cnt == 2: break # 한글, 영어 뜻까지만 찾고 for문 탈출
    
    return mean_data

In [93]:
# 테스트를 위해 작성된 코드
# 랜덤 함수를 이용하였기 때문에 해당 셀을 여러번 실행해도 다른 결과값이 나오고 이로부터 테스트 결과를 강건하게 확인 가능 
rand = random.randint(0, 100)
test_word = word_list[rand][0]
print(test_word)
print(get_mean(test_word))

plan
[' 1.계획하다 2.예정 3.플랜 4.구상 5.의도 ', ' a series of steps to be carried out or goals to be accomplished ']


# 4. English Word Test Program
해외 매체에서 크롤링한 단어로 시험을 보는 프로그램을 아래에서 최종 작성하였다. 
1. 한영, 영영 모드 중 선택하여 테스트 가능 
2. 한 문제 당 총 n번(chance)의 기회 제공
3. 기회 횟수가 3회 이하로 떨어질 경우 hint 제공

In [110]:
def show_hint(word, chance):
    chars = [c for c in word]                 # 단어를 문자 단위로 저장 (ex, words -> w o r d s)
    hcnt = int(len(chars) * (1 - 0.5/chance)) # 남은 기회(chance) 비율에 따라 힌트로 제공할 문자 수를 조정함 
    
    rand = random.sample(range(0, len(chars)-1), hcnt)
    for r in range(len(rand)): # 남은 기회 비율에 따라 랜덤으로 가릴 문자 선택 
        chars[rand[r]] = "_"
        
    return print(f'힌트: {" ".join(chars)}')

In [114]:
def word_test(WORDS_LIST, chance): 
    if len(WORDS_LIST) <= 0 : print("단어 데이터가 수집되어 있지 않습니다. 데이터를 수집하고 시작해주세요.") # error handling
    random.shuffle(WORDS_LIST)
    
    # 테스트 모드 선택 
    MODE = int(input("진행할 단어 테스트의 모드를 입력하세요. \n1) 한영 테스트 \t 2) 영영 테스트 \n\n "))
    # 단어 테스트 시작
    for i, word in enumerate(WORDS_LIST):
        mean = get_mean(word)[MODE-1]
        print("\n", mean) 
        CHANCE = chance
        # 기회(chance) 안에 정답을 맞추도록 구성
        while CHANCE > 0:
            print("*" * 80)
            user = input("뜻의 단어를 입력하세요(종료는 'Q' 입력) \n>>>>> ").strip()
            if user == "q" or user == "Q": 
                return 0
            if word.lower() == user.lower(): 
                print("정답!")
                break
            else:
                CHANCE -= 1
                if CHANCE == 0: 
                    print(f"정답은 {word} 입니다. \n\n")
                    break
                print("\n틀렸습니다. 남은 기회 ", str(CHANCE), "회..")
                if CHANCE <= 3: show_hint(word, CHANCE) # 기회가 3회 이하로 떨어질 경우 힌트 제공 

In [123]:
result = word_test(words, 5) # 한 문제 당 주어지는 기회는 직접 설정 가능
if result == 0 : print("\n\n프로그램을 종료합니다...")

진행할 단어 테스트의 모드를 입력하세요. 
1) 한영 테스트 	 2) 영영 테스트 

 1

  1.차별하다 2.식별하다 
********************************************************************************
뜻의 단어를 입력하세요(종료는 'Q' 입력) 
>>>>> discriminate
정답!

  1.표제 2.자막 3.타이틀 
********************************************************************************
뜻의 단어를 입력하세요(종료는 'Q' 입력) 
>>>>> title

틀렸습니다. 남은 기회  4 회..
********************************************************************************
뜻의 단어를 입력하세요(종료는 'Q' 입력) 
>>>>> index

틀렸습니다. 남은 기회  3 회..
힌트: _ a _ _ _ _ n
********************************************************************************
뜻의 단어를 입력하세요(종료는 'Q' 입력) 
>>>>> an

틀렸습니다. 남은 기회  2 회..
힌트: c _ _ _ _ _ n
********************************************************************************
뜻의 단어를 입력하세요(종료는 'Q' 입력) 
>>>>> caland

틀렸습니다. 남은 기회  1 회..
힌트: c a p _ _ _ n
********************************************************************************
뜻의 단어를 입력하세요(종료는 'Q' 입력) 
>>>>> caption
정답!

  굽다 
*************************************